In [ ]:
from eppy.modeleditor import IDF
from openstudio import energyplus, osversion, path
from database import ResstockDatabase

In [ ]:
database_filepath = 'database.sql'
resstock_database = ResstockDatabase(filepath=database_filepath)

# Build SQLite DB
***

In [ ]:
filters = {
    'in.geometry_building_type_recs':['Single-Family Detached'],
    'in.vacancy_status':['Occupied'],
    'in.resstock_county_id':['TX, Travis County']
}
kwargs = {
    'dataset_type':'resstock',
    'weather_data':'tmy3',
    'year_of_publication':2021,
    'release':1,
    'filters':filters
}
resstock_database.build(overwrite=True)
resstock_database.insert_dataset(**kwargs)

# E+ Simulation
***

In [ ]:
sim_data = resstock_database.query_table("""
SELECT
    bldg_id,
    osm,
    epw,
    ddy
FROM building_energy_performance_simulation_input
WHERE bldg_id = '40'
""")

# with open('debug/model.osm','w') as f:
#         f.write(sim_data['osm'].iloc[0])

# osm_string = sim_data['osm'].iloc[0]
# version_translator = osversion.VersionTranslator()
# osm_model = version_translator.loadModel('model_raw.osm').get()
# forward_translator = energyplus.ForwardTranslator()
# idf_model = forward_translator.translateModel(osm_model)
# _ = idf_model.save(path('model_raw.idf'),True)

In [ ]:
idd_filepath = '/Applications/EnergyPlus-9-6-0/PreProcess/IDFVersionUpdater/V9-6-0-Energy+.idd'
idf_filepath = 'model_raw.idf'
epw_filepath = '/Users/kingsleyenweye/Desktop/INTELLIGENT_ENVIRONMENT_LAB/tmy_management/USA_TX_Austin-Camp.Mabry.722544_TMY3.epw'
ddy_filepath = '/Users/kingsleyenweye/Desktop/INTELLIGENT_ENVIRONMENT_LAB/tmy_management/USA_TX_Austin-Camp.Mabry.722544_TMY3.ddy'
simulation_id = idf_filepath.split('.')[0]
IDF.setiddname(idd_filepath)
idf = IDF(idf_filepath,epw_filepath)

# preprocessing here
# ...

# # run energy simulation
# idf_version = idf.idfobjects['version'][0].Version_Identifier.split('.')
# idf_version.extend([0] * (3 - len(idf_version)))
# idf_version_str = '-'.join([str(item) for item in idf_version])
# options = {
#     'ep_version':idf_version_str,
#     'output_prefix':'eplusout',
#     'output_suffix':'C',
#     'output_directory':'model_raw',
#     'readvars':False,
#     'expandobjects':False,
#     'verbose':'q',
# }
# idf.run(**options)